# Advent of Code: Day 3

## Part One

--- Day 3: Gear Ratios ---
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [ ]:
from dataclasses import dataclass, field
import logging
import re
from typing import List, Set, Union

In [ ]:
# Set up logging for Jupyter Notebook
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(format=' %(asctime)s - %(levelname)s - %(message)s')

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [ ]:
@dataclass
class PartSymbol:
    s: str
    start: int
    end: int
    row: int
    connections: Set['PartSymbol'] = field(default_factory=set, init=False)

    def __repr__(self):
        if self.is_gear():
            return f'PartSymbol(s="{self.s}", row={self.row}, start={self.start}, end={self.end}, connections={self.connections})'
        return f'PartSymbol(s="{self.s}", row={self.row}, start={self.start}, end={self.end}, connections={[c.to_string() for c in self.connections]}'

    def to_string(self):
        return f'PartSymbol(s="{self.s}", row={self.row}, start={self.start}, end={self.end}, connections={"[]" if len(self.connections) == 0 else "[...]"})'
        
    def __hash__(self):
        return hash((self.s, self.start, self.end, self.row))
    
    def is_number(self) -> bool:
        return self.s.isdigit()

    def get_number(self) -> Union[int, None]:
        if self.is_number():
            return int(self.s)
        return None

    def connect_to(self, other: 'PartSymbol'):
        self.connections.add(other)

    def clear_connections(self):
        self.connections = set([])

    def is_connected(self) -> bool:
        return len(self.connections) > 0

    def is_gear(self) -> bool:
        return self.s == '*' and len(self.connections) == 2

    def gear_value(self) -> int:
        if self.is_gear():
            numbers = [c.get_number() for c in self.connections]
            if len(numbers) != 2:
                raise ValueError(f'Expected only two connections, instead got {self.connections}')
            return numbers[0] * numbers[1]


@dataclass
class PartSymbolRow:
    row: int
    symbols: List[PartSymbol] = field(default_factory=list, init=False)

    def append(self, symbol: PartSymbol):
        self.symbols.append(symbol)


@dataclass
class Schematic:
    part_symbol_rows: List[PartSymbolRow] = field(default_factory=list, init=False)

    def append(self, row: PartSymbolRow):
        self.part_symbol_rows.append(row)

    def connect(self):
        """Connect number symbols to nearby character symbols.
        """
        self.disconnect()
        for row, part_symbol_row in enumerate(self.part_symbol_rows):
            row_lower_bound: int = max([row - 1, 0])
            row_upper_bound: int = min([row + 1, len(self.part_symbol_rows)])
            for other_row in range(row_lower_bound, row_upper_bound):
                # logger.info(f'row: {row}, other_row: {other_row}')
                part_symbol_row: PartSymbolRow = self.part_symbol_rows[row]
                other_part_symbol_row: PartSymbolRow = self.part_symbol_rows[other_row]
                for part_symbol in part_symbol_row.symbols:
                    # logger.info(f'considering {part_symbol}')
                    for other_part_symbol in other_part_symbol_row.symbols:
                        if part_symbol.is_number() and other_part_symbol.is_number():
                            continue
                        elif not part_symbol.is_number() and not other_part_symbol.is_number():
                            continue
                        if other_part_symbol.end >= part_symbol.start and other_part_symbol.start <= part_symbol.end:
                            part_symbol.connect_to(other_part_symbol)
                            other_part_symbol.connect_to(part_symbol)
                            # if part_symbol.is_number():
                            #     part_symbol.connect_to(other_part_symbol)
                            # else:
                            #     other_part_symbol.connect_to(part_symbol)
                            # logger.info(f'connected {part_symbol} to {other_part_symbol}')
                        else:
                            # logger.info(f'no connection from {part_symbol} to {other_part_symbol}')
                            pass

    def disconnect(self):
        """Remove all the links between the symbols."""
        pass

    def get_connected_parts(self) -> List[PartSymbol]:
        parts: List[PartSymbol] = []
        for part_symbol_row in self.part_symbol_rows:
            for part_symbol in part_symbol_row.symbols:
                if part_symbol.is_connected():
                    parts.append(part_symbol)
        return parts


def get_input():
    with open('20231203-input.txt', 'r') as f:
        return [line.strip() for line in f]


def parse_raw_schematic(raw_schematic: List[str]) -> Schematic:
    schematic: Schematic = Schematic()
    number_re_pattern: str = r'(?P<number>\d+)'
    number_p = re.compile(number_re_pattern)
    symbol_re_pattern: str = r'(?P<symbol>[^0-9.])'
    symbol_p = re.compile(symbol_re_pattern)
    for row, line in enumerate(raw_schematic):
        part_symbol_row = PartSymbolRow(row=row)
        number_match_list = number_p.finditer(line)
        for match in number_match_list:
            s = match.group('number')
            start = match.start()
            end = match.end()
            part_symbol = PartSymbol(s=s, start=start, end=end, row=row)
            part_symbol_row.append(part_symbol)
            # logger.info(part_symbol)
        symbol_match_list = symbol_p.finditer(line)
        for match in symbol_match_list:
            s = match.group('symbol')
            start = match.start()
            end = match.end()
            part_symbol = PartSymbol(s=s, start=start, end=end, row=row)
            part_symbol_row.append(part_symbol)
            # logger.info(part_symbol)
        schematic.append(part_symbol_row)
        # logger.info(part_symbol_row)
        # if row >= 5:
        #     break
    schematic.connect()
    return schematic


def solution_part1():
    raw_schematic: List[str] = get_input()
    schematic: Schematic = parse_raw_schematic(raw_schematic)
    numbers: List[int] = []
    # logger.info('connected parts')
    for part in schematic.get_connected_parts():
        # logger.info(part)
        if part.is_number():
            numbers.append(part.get_number())
    return sum(numbers)


solution_part1()

--- Part Two ---
The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

In [ ]:
def solution_part2():
    raw_schematic: List[str] = get_input()
    schematic: Schematic = parse_raw_schematic(raw_schematic)
    gear_values: List[int] = []
    for part in schematic.get_connected_parts():
        # logger.info(part)
        if part.is_gear():
            value: int = part.gear_value()
            # logger.info(f'{part}, value: {value}')
            gear_values.append(value)
        # elif part.s == '*':
        #     logger.info(f'Not a gear: {part}')
    return sum(gear_values)


solution_part2()